<a href="https://colab.research.google.com/github/Aryabhatt-O/Computational-Geometry/blob/main/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PCA**

In [ ]:


import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA


iris = datasets.load_iris()

X = iris.data
y = iris.target
target_names = iris.target_names

pca = PCA(n_components=3)
X_r = pca.fit(X).transform(X)





plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')


plt.show()

# **Dataset**

In [ ]:
# print(X_r)
# len(X_r)
import numpy as np
for i,j in enumerate(X_r):
  print("point",[i],"-->",X_r[i])


In [ ]:
import numpy as np
X_r_2d = np.delete(X_r, 2, 1)
for i,j in enumerate(X_r_2d):
  print("point",[i],"-->",X_r_2d[i])
 

# **10% of Total used as Centroid**

In [ ]:
import numpy as np
centroid = X_r_2d[np.random.randint(X_r_2d.shape[0], size=15), :]
print(centroid)

if len(np.unique(centroid, axis=0)) == len(centroid) :
  print("GO AHEAD")
else:
  print("RE-SHUFFLE")

# len(centroid)
# len(np.unique(centroid, axis=0))

### **These are the numbers of Original X_r_2d from which Centriods are taken**

In [ ]:
import numpy as np
from scipy import spatial

def find_nearest(big_array,small_array,tolerance):
    tree_big=spatial.cKDTree(big_array)
    tree_small=spatial.cKDTree(small_array)
    return tree_small.query_ball_tree(tree_big,r=tolerance)

higher_dim_index = []
higher_dim_index = find_nearest(X_r_2d,centroid,0)
print(higher_dim_index)

# **Rest Test Data**(Which are different from centroid points)

In [ ]:
rest_data = X_r_2d[np.random.choice(X_r_2d.shape[0], 135, replace=False), :]
# print(rest_data)

if len(np.unique(rest_data, axis=0)) == 135:
  print("GO AHEAD")
else:
  print("RE-SHUFFLE")

# **Centroid points are given to create Voronoi upon which Rest Test_data is distributed**

In [ ]:
from random import *
import heapq
import numpy as np




# For checking from voronoi

from random import *
import random
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
import numpy as np
vor = Voronoi(centroid,qhull_options='Qbb Qc Qx')
voronoi_plot_2d(vor,show_vertices = True)

#for plotting my friends location 
x, y = rest_data.T
plt.plot(x,y, 'ko')

for region in vor.regions:
    if not -1 in region:
        polygon = [vor.vertices[i] for i in region]
        plt.fill(*zip(*polygon))



# **Numbering Voronoi Cell**

In [ ]:

for i,a in enumerate(centroid):
   print("Centriod index ---> ",a,"corresponding vor_cell -->",vor.point_region[i])
   


In [ ]:
from scipy.spatial import cKDTree
voronoi_kdtree = cKDTree(centroid)
test_point_dist, test_point_regions = voronoi_kdtree.query(rest_data)
l= test_point_regions
m = test_point_dist
print(l)



# **{Vor_CELL number : index of points inside that region referred to rest_points}**

In [ ]:
from collections import defaultdict
cell_wise_point = []
def list_duplicates(l):
    tally = defaultdict(list)
    for i,item in enumerate(l):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items() 
                            if len(locs)>1)

for dup in sorted(list_duplicates(l)):
    # print(dup)
    cell_wise_point.append(dup)

c_w_cord = dict(cell_wise_point)
print(c_w_cord)

In [ ]:
vor_reg = []
vor_inside_points = []
for key in c_w_cord.keys():
    for i in c_w_cord.get(key):
        print('cell',key,'-->','co-ordinates',rest_data[i])
        vor_reg.append(key)
        vor_inside_points.append(rest_data[i])


print(vor_reg,vor_inside_points)

type(vor_inside_points)


# **{Vor_CELL number : Co-Ordinates of points inside that region}**

In [ ]:
vor_inside_point_coOrdinates = {}
i = iter(vor_reg)
j = iter(vor_inside_points)
k = list(zip(i, j))
for (x,y) in k:
    if x in vor_inside_point_coOrdinates:
        vor_inside_point_coOrdinates[x] = vor_inside_point_coOrdinates[x] , y # whatever function needs to be to combine them
    else:
        vor_inside_point_coOrdinates[x] = y

vor_inside_point_coOrdinates

# **{Vor_CELL number : sum of Co-Ordinates of points regionwise}**

In [ ]:
vor_inside_point_sum = {}
i = iter(vor_reg)
j = iter(vor_inside_points)
k = list(zip(i, j))
for (x,y) in k:
    if x in vor_inside_point_sum:
        vor_inside_point_sum[x] = vor_inside_point_sum[x] + y # whatever function needs to be to combine them
    else:
        vor_inside_point_sum[x] = y

vor_inside_point_sum

# **{Vor_CELL number : Total No of points  of point regionWise}**

In [ ]:
unique, counts = np.unique(l, return_counts=True)
vor_inside_point_count = dict(zip(unique, counts))
print(vor_inside_point_count)

# **Pseudo-Centroid**


In [ ]:
{k: vor_inside_point_sum[k]/vor_inside_point_count[k] for k in vor_inside_point_sum.keys() & vor_inside_point_count}


In [ ]:
Pseudo_centroid = []
Pseudo_centroid_dict = {k: vor_inside_point_sum[k]/vor_inside_point_count[k] for k in vor_inside_point_sum.keys() & vor_inside_point_count}
Pseudo_centroid = Pseudo_centroid_dict.values()
Pseudo_centroid_list = list(Pseudo_centroid) 

Pseudo_centroid_list


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
distance_matrix= cosine_similarity(Pseudo_centroid_list,Pseudo_centroid_list)
print(distance_matrix)

# **DATAFRAME of CosineMatrix of Pseudo-Centroids**

In [ ]:
import pandas as pd
data_frame = pd.DataFrame(distance_matrix)

print(data_frame)

In [ ]:
data_frame["SUM of roWs"] = data_frame.sum(axis=1)

print(data_frame)

# **TOP 3 Pseudo-Centroid**

In [ ]:
k_vector = data_frame.nsmallest(3, 'SUM of roWs')
k_vector

In [ ]:
index = k_vector.index
index.name = "Index"
k_vector

### ***Eucledian_way***

In [ ]:
from scipy.spatial import distance
euclidean_Pseudo_matrix = distance.cdist(Pseudo_centroid_list,Pseudo_centroid_list, 'euclidean')
euclidean_Pseudo_data_frame = pd.DataFrame(euclidean_Pseudo_matrix)

print(euclidean_Pseudo_data_frame)

In [ ]:
euclidean_Pseudo_data_frame["SUM of roWs"] = euclidean_Pseudo_data_frame.sum(axis=1)

print(euclidean_Pseudo_data_frame)

In [ ]:
eu_vector = data_frame.nlargest(3, 'SUM of roWs')
eu_vector

In [ ]:
index = eu_vector.index
index.name = "Index"
eu_vector

# **Translating to Higher-Dimension**

### ***Step-1***

In [ ]:
final_pseudo_vectors = []
list1 =  list((k_vector.index))
for i in list1:
  final_pseudo_vectors.append(Pseudo_centroid_list[i])


final_pseudo_vectors

In [ ]:
final_pseudo_vectors_eu = []
list1 =  list((eu_vector.index))
for i in list1:
  final_pseudo_vectors_eu.append(Pseudo_centroid_list[i])


final_pseudo_vectors_eu

### ***Step-2***

In [ ]:
from scipy.spatial import distance
pseudo_to_original_centroid_mapping = distance.cdist(final_pseudo_vectors, centroid, 'euclidean')
pseudo_to_original_centroid_mapping_data_frame = pd.DataFrame(pseudo_to_original_centroid_mapping)

print(pseudo_to_original_centroid_mapping_data_frame)

In [ ]:
from scipy.spatial import distance
pseudo_to_original_centroid_mapping_eu = distance.cdist(final_pseudo_vectors_eu, centroid, 'euclidean')
pseudo_to_original_centroid_mapping_data_frame_eu = pd.DataFrame(pseudo_to_original_centroid_mapping_eu)

print(pseudo_to_original_centroid_mapping_data_frame_eu)

### ***Step-3***

In [ ]:
mapping =  pseudo_to_original_centroid_mapping_data_frame.idxmin(axis=1)
mapping_data_frame = pd.DataFrame(mapping)
index = mapping_data_frame.index
index.name = "Pseudo-centroid"
mapping_data_frame.columns = ['Corresponding_original_Centroid']
mapping_data_frame


In [ ]:
mapping_eu =  pseudo_to_original_centroid_mapping_data_frame_eu.idxmin(axis=1)
mapping_data_frame_eu = pd.DataFrame(mapping_eu)
index = mapping_data_frame_eu.index
index.name = "Pseudo-centroid"
mapping_data_frame_eu.columns = ['Corresponding_original_Centroid']
mapping_data_frame_eu


### ***Step-4***

In [ ]:
final_centroid_vectors = []
list2 =  list((mapping_data_frame.Corresponding_original_Centroid))
for i in list2:
  final_centroid_vectors.append(centroid[i])


final_centroid_vectors

In [ ]:
final_centroid_vectors_eu = []
list2_eu =  list((mapping_data_frame_eu.Corresponding_original_Centroid))
for i in list2_eu:
  final_centroid_vectors_eu.append(centroid[i])


final_centroid_vectors_eu

### ***Step-5***

In [ ]:
high_dim_centroid = []
for i in list2:
  a = higher_dim_index[i]
  high_dim_centroid.append(X_r[a])

high_dim_centroid

In [ ]:
high_dim_centroid_eu = []
for i in list2_eu:
  a_eu = higher_dim_index[i]
  high_dim_centroid_eu.append(X_r[a])

high_dim_centroid_eu

# ***END***

# **This depicts Which voronoi cell contains how many test data points**

{*Cell No* : **No of points assosiated in it**}

In [ ]:

unique, counts = np.unique(l, return_counts=True)
count = dict(zip(unique, counts))
print(count)

# **Region Vertices**

In [ ]:
for i in (range(len(centroid))):
  print("points",[i], "=", centroid[i])
  print( 'and corresponding voronoi region','-->',[vor.point_region[i]],)
  print("vertices for region for point",i,"=", vor.regions[vor.point_region[i]])
  print("coordinate of Voronoi vertices associated with point", i,":")
  print(vor.vertices[vor.regions[vor.point_region[i]]])
  

In [ ]:
print(vor.point_region)

In [ ]:
for i, reg in enumerate(vor.regions):

    print('Region:', i)
    print('Indices of vertices of Voronoi region:', reg)
    print('Associated point:', rest_data[reg], '\n')

# **Area and Total Number of Points**

In [ ]:
def PolyArea2D(pts):
    lines = np.hstack([pts,np.roll(pts,-1,axis=0)])
    area = 0.5*abs(sum(x1*y2-x2*y1 for x1,y1,x2,y2 in lines))
    return area

cell_area = []
number_of_points = []
corresponding_centroid = []

for i in range(len(centroid)):
  print("points",[i], "=", centroid[i],'and no of  points inside it','-->',count[vor.point_region[i]])
  corresponding_centroid.append(centroid[i])
  number_of_points.append(count[vor.point_region[i]])
  print('Area-->',PolyArea2D(vor.vertices[vor.regions[vor.point_region[i]]]))
  cell_area.append(PolyArea2D(vor.vertices[vor.regions[vor.point_region[i]]]))
 

print(len(cell_area),len(number_of_points))



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
distance_matrix= cosine_similarity(centroid,centroid)
print(distance_matrix)

type(centroid)

In [ ]:

points_centroid = dict(zip(number_of_points,corresponding_centroid))
print(points_centroid)
# type(points_centroid)


In [ ]:
import operator
max(points_centroid.items(), key=operator.itemgetter(0))[1]

In [ ]:
max = []
for i in (sorted((number_of_points),reverse = True)):
  max.append(points_centroid.get(i))
  print(points_centroid.get(i))

# **(No of points, Area)**

In [ ]:
l = zip(number_of_points,cell_area)
points_area = (list(l))
print((points_area))

# **Density Array**

In [ ]:
from operator import truediv
density_value = list(map(truediv,number_of_points,cell_area))
sorted_density_list = (sorted((density_value),reverse = True))
# print(sorted_density_list)
print((density_value))

# final_centroid1= np.argmax(density_value)


# **Top density wise Centroid point index**



In [ ]:
index = sorted(range(len(density_value)),reverse = True, key=density_value.__getitem__)
print(index)

# **Top Centroid List**

In [ ]:
rank_wise_centroid = []
for i in range(0,len(index)):
  rank_wise_centroid.append(centroid[index[i]])
# print(rank_wise_centroid)


for i,j in enumerate(rank_wise_centroid):
  print("rank",i,"--->",j)

# **1st CENTROID**

In [ ]:
final_centroid1 = rank_wise_centroid[0]
print("THIS IS CENTROID 1:-",final_centroid1)

# **Next CENTROIDs Tuning**

In [ ]:
bool_arr0 = []
from sklearn.metrics.pairwise import cosine_similarity
distance_matrix= cosine_similarity(rank_wise_centroid,rank_wise_centroid)
# print(distance_matrix[0])

for i in range(len(distance_matrix[0])):
  if distance_matrix[0][i] <= 0.5:
    bool_arr0.append(True)
  else:
    bool_arr0.append(False)
print(bool_arr0)


distance_matrix_checker_index = []
possible_centroid2 = []
for i in range(len(bool_arr0)):
  if bool_arr0[i] == True:
    possible_centroid2.append(rank_wise_centroid[i])
    distance_matrix_checker_index.append(i)
if len((possible_centroid2)) == 0:
  print("NEXT ITERATION")
else:
  final_centroid2 = possible_centroid2[0]
  print(" IS THIS CENTROID 2?:-",final_centroid2, "next will check in destance_matrix",[distance_matrix_checker_index[0]],":-")
  # print(distance_matrix_checker_index)


# *COLLECTED CENTROIDS FOR THIS MODEL*





### **clus_1 = [3.49992004,0.4606741]**


---


### **clus_2 = [-2.62614497,0.16338496]**

---


### **clus_3 = [ 0.94473373, -0.54314555]**

In [ ]:
bool_arr_five = []

# print(distance_matrix[0])

for i in range(len(distance_matrix[5])):
  if distance_matrix[5][i] <= 0.5:
    bool_arr_five.append(True)
  else:
    bool_arr_five.append(False)
print(bool_arr_five)


# # distance_matrix_checker_index = []
# possible_centroid2 = []
# for i in range(len(bool_arr0)):
#   if bool_arr0[i] == True:
#     possible_centroid2.append(rank_wise_centroid[i])
#     distance_matrix_checker_index.append(i)
# if len((possible_centroid2)) == 0:
#   print("NEXT ITERATION")
# else:
#   final_centroid2 = possible_centroid2[0]
#   print(" IS THIS CENTROID 2?:-",final_centroid2)
#   # print(distance_matrix_checker_index)


In [ ]:
# print(distance_matrix)
# print(distance_matrix[0])
print(distance_matrix[5])
print(distance_matrix[1])



In [ ]:
vector1 = [1, 2]
vector2 = [3, 2]

cosine_similarity = 1 - spatial.distance.cosine(vector1, vector2)
print(cosine_similarity)

In [ ]:
import scipy
from scipy import sparse
sparse_matrix= scipy.sparse.csr_matrix(centroid)
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

distance_matrix= pairwise_distances(sparse_matrix, metric="cosine")

print(distance_matrix)



# **Top 3 Crowded Cell**

In [ ]:

from heapq import nlargest,nsmallest
# Initialize N
N = 5
res = nlargest(N, count, key = count.get)

print("The top 3 value cells are " + str(res))


# **Centroid Point of those 3 cell**

# **Distance Matrix**

In [ ]:
from scipy.spatial import distance
distance.cdist(cell_centr_tracker, cell_centr_tracker, 'euclidean')

In [ ]:
!pip install scipy
import scipy
from scipy import sparse
sparse_matrix= scipy.sparse.csr_matrix(cell_centr_tracker)

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

distance_matrix= pairwise_distances(sparse_matrix, metric="cosine")
print(distance_matrix)